1. Parse, clean, and organize the Jeopardy! question data file to train a Naive Bayesian classifier.

Just as we have built a classifier above, your aim here is to make sense of the data presented and create a binary classifier ("high value" and "low value," based on the points available for each) for questions. Despite the large number of questions, this is an extraordinarily difficult classification problem. Consider it as a human coder: how often could you tell those questions that are "easy" versus "hard"? The degree to which you are successful in this is largely based on your own contextual knowledge--indeed, you might be tempted to classify questions you know the answer to as "easy" and those you do not as "hard." The computer doesn't know the answers to any of these.

For that reason, do not be discouraged if your classifier does not perform well. This constitutes an especially difficult problem for a simple classifier to solve.

In [1]:
import json

In [2]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
import pandas as pd
from string import punctuation
import re
import time
from datetime import datetime

In [3]:
from sklearn.naive_bayes import GaussianNB

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

In [5]:
import numpy as np

In [6]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/malirandolph/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [7]:
import pandas as pd

In [8]:
nltk.download('stopwords')  

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/malirandolph/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

import os
import re

from sklearn.metrics import classification_report

In [187]:
with open('/Users/malirandolph/Downloads/jeopardy.json', 'r', newline='\r\n') as f:
    data = json.load(f)

In [188]:
# converts json file to csv
df = pd.DataFrame(data)

In [189]:
df.to_csv('modfive.csv')

In [190]:
df = pd.read_csv('modfive.csv', index_col=0)

In [90]:
len(df)

216930

# removing dollar signs from value column so we can convert inputs to floats
df['value'] = df['value'].map(str).str.replace('$','').str.strip()

In [191]:
# assigning the rounds a value
df['round'] = df['round'].apply(lambda x: 1 if x=="Jeopardy!" else 0 if x=="Double Jeopardy!" else -1).astype(str)

In [192]:
df["question"] = df["question"].replace(",", "", regex=True)

In [193]:
df["question"] = df["question"].replace(":", "", regex=True)

In [194]:
df["question"] = df["question"].replace(";", "", regex=True)

In [195]:
df['question'] = df['question'].str.replace('\d+', '')

/var/folders/df/z6m4t_kj47s4qtpl7pfk3fgw0000gn/T/ipykernel_59945/2485659254.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['question'] = df['question'].str.replace('\d+', '')


In [196]:
df['question'] = df['question'].replace(to_replace=r'^https?:\/\/.*[\r\n]*',value='',regex=True)

In [197]:
df['question'] = df['question'].str.rstrip('href')

In [198]:
df['question'] = df['question'].replace('"', '', regex=True)

removes all punctuation from dataframe

In [199]:
df['question'] = df['question'].str.replace(r'[^\w\s]+', '')

/var/folders/df/z6m4t_kj47s4qtpl7pfk3fgw0000gn/T/ipykernel_59945/3651572173.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['question'] = df['question'].str.replace(r'[^\w\s]+', '')


In [54]:
df = df[['round', 'question']]

In [124]:
df.head(10)

,category,air_date,question,value,answer,round,show_number
0,HISTORY,2004-12-31,For the last years of his life Galileo was un...,$200,Copernicus,1,4680
1,ESPN's TOP 10 ALL-TIME ATHLETES,2004-12-31,No Olympian football star at Carlisle Indian...,$200,Jim Thorpe,1,4680
2,EVERYBODY TALKS ABOUT IT...,2004-12-31,The city of Yuma in this state has a record av...,$200,Arizona,1,4680
3,THE COMPANY LINE,2004-12-31,In live on The Art Linkletter Show this compa...,$200,McDonald\'s,1,4680
4,EPITAPHS & TRIBUTES,2004-12-31,Signer of the Dec of Indep framer of the Const...,$200,John Adams,1,4680
5,3-LETTER WORDS,2004-12-31,In the title of an Aesop fable this insect sha...,$200,the ant,1,4680
6,HISTORY,2004-12-31,Built in BC to link Rome the South of Italy ...,$400,the Appian Way,1,4680
7,ESPN's TOP 10 ALL-TIME ATHLETES,2004-12-31,No steals for the Birmingham Barons steals ...,$400,Michael Jordan,1,4680
8,EVERYBODY TALKS ABOUT IT...,2004-12-31,In the winter of a record inches of snow fel...,$400,Washington,1,4680
9,THE COMPANY LINE,2004-12-31,This housewares store was named for the packag...,$400,Crate & Barrel,1,4680


In [200]:
df['tokenized_text'] = df['question'].apply(word_tokenize) 

In [176]:
#convert each word into a column
listss = pd.DataFrame(df.tokenized_text.tolist())

In [179]:
listss = listss.astype(str)

In [180]:
listss

,0,1,2,3,4,5,6,7,8,9,...,121,122,123,124,125,126,127,128,129,130
0,For,the,last,years,of,his,life,Galileo,was,under,...,None,None,None,None,None,None,None,None,None,None
1,No,Olympian,football,star,at,Carlisle,Indian,School,MLB,seasons,...,None,None,None,None,None,None,None,None,None,None
2,The,city,of,Yuma,in,this,state,has,a,record,...,None,None,None,None,None,None,None,None,None,None
3,In,live,on,The,Art,Linkletter,Show,this,company,served,...,None,None,None,None,None,None,None,None,None,None
4,Signer,of,the,Dec,of,Indep,framer,of,the,Constitution,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216925,This,Puccini,opera,turns,on,the,solution,to,riddles,posed,...,None,None,None,None,None,None,None,None,None,None
216926,In,North,America,this,term,is,properly,applied,to,only,...,None,None,None,None,None,None,None,None,None,None
216927,In,Penny,Lane,where,this,Hellraiser,grew,up,the,barber,...,None,None,None,None,None,None,None,None,None,None
216928,From,Ft,Sill,Okla,he,made,the,plea,Arizona,is,...,None,None,None,None,None,None,None,None,None,None


In [177]:
testdf = listss.dropna(axis=1,how='all', inplace=False)

In [181]:
df = df[['round']]

In [21]:
df = df[['round', 'tokenized_text']]

In [72]:
df

,round
0,1
1,1
2,1
3,1
4,1
...,...
216925,0
216926,0
216927,0
216928,0


In [182]:
X_train, X_test, y_train, y_test = train_test_split(listss, df)

In [183]:
tfidf_vectorizer = TfidfVectorizer(use_idf=True)

In [184]:
x_train_tf = tfidf_vectorizer.fit_transform(X_train)

AttributeError: 'int' object has no attribute 'lower'

In [319]:
x_train_tf

<2x2 sparse matrix of type '<class 'numpy.float64'>'
	with 2 stored elements in Compressed Sparse Row format>

In [286]:
naive_bayes = MultinomialNB()

In [312]:
naive_bayes.fit(x_train_tf, y_train)

ValueError: Found input variables with inconsistent numbers of samples: [2, 162697]

countvectorizer = CountVectorizer(analyzer= 'word', stop_words='english')

In [251]:
tfidfvectorizer = TfidfVectorizer(analyzer='word',stop_words= 'english')

In [252]:
df['tokenized_text'] = df['tokenized_text'].map(' '.join)

In [205]:
tfidf_vectors = tfidf_vectorizer.fit_transform(df['tokenized_text'])

In [253]:
df = df[['round', 'tokenized_text']].T

In [255]:
as_array = tfidf_vectors.toarray()

In [206]:
feature_names = tfidf_vectorizer.get_feature_names()